# Подключаем библиотеки

In [1]:
import json
import numpy as np
import os
import pandas as pd
import pyspark
import pyspark.pandas as ps
import random

from pyspark.sql import SparkSession
from pyspark.sql.column import Column
from pyspark.sql.column import _to_java_column
from pyspark.sql.column import _to_seq
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

/usr/local/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


# Читаем данные

Оптимизация

1. На видео и комментарии вешаем bucketing по video id - по этому полю происходит join чаще всего, и подобный подход позволяет избежать практически всех Exchange за исключением случаев, например, с groupby по другой колонке.
2. На категории вешаем `broadcast` - маленький справочник
3. Другие вариант оптимизации нерентабельны для соления нет значительного перекоса, фильтр Блума практически ничего не отфильтрует - джоин видео и комментариев не так много строк отбрасывает

In [2]:
spark = SparkSession.builder.master('local') \
    .config('spark.sql.autoBroadcastJoinThreshold', 0) \
    .config('spark.sql.adaptive.enabled', 'false') \
    .getOrCreate()

In [3]:
# videos = spark.read\
#     .option('header', 'true')\
#     .option('inferSchema', 'true')\
#     .csv('datasets/USvideos.csv')

videos_raw = spark.read\
    .option('header', 'true')\
    .option('inferSchema', 'true')\
    .csv('datasets/USvideos.csv')
videos_raw.write\
    .bucketBy(128, 'video_id')\
    .saveAsTable('USvideos', format = 'csv', mode = 'overwrite')
videos = spark.table('USvideos')

In [4]:
videos_sdim_df = videos\
    .withColumn('rnk', row_number()\
                .over(Window.partitionBy('video_id').orderBy(col('date').desc(), col('views').desc()))\
                )\
                .filter(col('rnk') == 1)\
                .drop(col('rnk'))

In [5]:
comments_schema = StructType([ \
    StructField('video_id', StringType(), True), \
    StructField('comment_text', StringType(), True), \
    StructField('likes', IntegerType(), True), \
    StructField('replies', IntegerType(), True)])

# comments = spark.read.option('header', 'true').option('mode', 'DROPMALFORMED').schema(comments_schema).csv('datasets/UScomments.csv')

comments_raw = spark.read.option('header', 'true').option('mode', 'DROPMALFORMED').schema(comments_schema).csv('datasets/UScomments.csv')

comments_raw.write \
    .bucketBy(128, 'video_id') \
    .saveAsTable('USComments', fprmat = 'csv', mode = 'overwrite')

comments = spark.table('USComments')

# Считаем рейтинг видео

In [6]:
@udf(returnType = DoubleType())
def calculate_video_score(views: int, likes: int, dislikes: int,comment_likes_count: int):
    return\
        int(likes or 0)\
        + random.random() * int(views or 0) * 0.2\
        - 2 * int(dislikes or 0)\
        + int(comment_likes_count or 0) * 0.01


In [7]:
@pandas_udf(DoubleType(),PandasUDFType.SCALAR)
def calculate_video_score(views, likes, dislikes,comment_likes_count):
    return\
        likes.fillna(0)\
        + np.random.rand(*views.shape) * views.fillna(0) * 0.2\
        - 2 * dislikes.fillna(0)\
        + comment_likes_count.fillna(0) * 0.01


/usr/local/spark/python/pyspark/sql/pandas/functions.py:399: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [8]:
video_comments_figures = comments.groupBy('video_id')\
    .agg(\
        sum('likes').alias('comment_likes_count'),\
        sum('replies').alias('comment_replies_count')\
        )

In [9]:
scored_video = videos_sdim_df.alias('videos')\
    .join(video_comments_figures, videos_sdim_df.video_id == video_comments_figures.video_id, 'left')\
    .select([
        'videos.*',
        col('comment_likes_count')
    ])\
    .withColumn(\
        'video_score',\
        calculate_video_score(\
            col('views'),\
            col('likes'),\
            col('dislikes'),\
            col('comment_likes_count')\
        )\
    )
scored_video.show()

+-----------+--------------------+--------------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+-------------------+------------------+
|   video_id|               title|       channel_title|category_id|                tags|  views| likes|dislikes|comment_total|      thumbnail_link| date|comment_likes_count|       video_score|
+-----------+--------------------+--------------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+-------------------+------------------+
|-LoSw4o2zDQ|How to Make Pushe...|    kawaiisweetworld|         26|recipe|recipes|ho...|  62600|  4715|      29|          483|https://i.ytimg.c...|14.10|                567|14304.832808630792|
|2Hz4sDjXPQY|Kingsman: The Gol...|    20th Century Fox|          1|Puppy|Spy Movies|...| 288222|  2003|      77|          208|https://i.ytimg.c...|21.09|               3692| 50462.12428286221|
|2YGrrsKs-Xg|Jimmy Kimmel’s FU...| 

# Собираем медиану рейтинга по категории

In [10]:
@pandas_udf(DoubleType(), PandasUDFType.GROUPED_AGG)
def pandas_mean(x):
    return x.mean()


In [11]:
categories_json = json.load(open('datasets/US_category_id.json'))
[x.update(x['snippet']) for x in categories_json['items']]

# categories_df = spark.read.json(\
#         spark.sparkContext.parallelize(categories_json['items']).map(lambda x: json.dumps(x))
#     ).select(['id', 'title'])\
#     .withColumnRenamed('title', 'category_title')

categories_df = broadcast(spark.read.json(\
        spark.sparkContext.parallelize(categories_json['items']).map(lambda x: json.dumps(x))
    ).select(['id', 'title'])\
    .withColumnRenamed('title', 'category_title'))

In [12]:
categories_score_df =  scored_video\
    .join(\
        categories_df,\
        scored_video.category_id == categories_df.id,\
        'left')\
    .groupBy('category_title')\
    .agg(
        pandas_mean(scored_video.video_score).alias('mean_score')
    )
categories_score_df.show()


+--------------------+------------------+
|      category_title|        mean_score|
+--------------------+------------------+
|               Shows|1149.7542397083937|
|           Education|64891.244558535014|
|              Gaming| 79795.06899696293|
|       Entertainment|136762.90155528876|
|     Travel & Events|53936.921381365726|
|Science & Technology|103083.25274564735|
|              Sports| 84678.97337789509|
|       Howto & Style| 80119.25228595799|
|Nonprofits & Acti...| 84960.11181624637|
|    Film & Animation| 114783.3445881312|
|      People & Blogs| 125965.0905301521|
|     News & Politics|48095.048304826116|
|      Pets & Animals|  78373.5449328473|
|    Autos & Vehicles|  85864.2600207591|
|               Music|226985.87508961212|
|              Comedy|184678.30789331047|
+--------------------+------------------+



In [13]:
categories_score_psdf = scored_video\
                .join(\
                    categories_df,\
                    scored_video.category_id == categories_df.id,\
                    'left')\
                .select([
                    categories_df.category_title,
                    scored_video.video_score
                ]).pandas_api()\
                .groupby(['category_title'])['video_score'].mean()
categories_score_psdf.to_frame().to_spark(index_col = 'category_title').show()

+--------------------+------------------+
|      category_title|       video_score|
+--------------------+------------------+
|               Shows|152.44155257191852|
|           Education|  68601.3536633228|
|              Gaming|114232.29525004755|
|       Entertainment|130068.50803116577|
|     Travel & Events| 42764.20499285154|
|Science & Technology|107895.05077725809|
|              Sports| 60017.79673221497|
|       Howto & Style| 84084.64206916135|
|Nonprofits & Acti...|  76852.2341191004|
|    Film & Animation|135470.48532708985|
|      People & Blogs|117692.51739586651|
|     News & Politics|49403.226958063584|
|      Pets & Animals| 73842.42585390672|
|    Autos & Vehicles| 64603.52009777978|
|               Music|  201868.490963744|
|              Comedy|178972.39863292963|
+--------------------+------------------+



In [14]:
categories_score_psdf_full = ps.merge(
    scored_video.pandas_api(),
    categories_df.pandas_api(),
    how = 'inner',
    left_on = 'category_id',
    right_on = 'id').groupby(['category_title'])['video_score'].mean()
categories_score_psdf_full.to_frame().to_spark(index_col = 'category_title').show()

+--------------------+------------------+
|      category_title|       video_score|
+--------------------+------------------+
|               Shows|1273.5684870669998|
|           Education|  76862.8979274907|
|              Gaming|101646.47955064183|
|       Entertainment|130674.41466660566|
|     Travel & Events| 67133.58294280886|
|Science & Technology| 91794.05718070792|
|              Sports| 65767.65705043857|
|       Howto & Style| 79194.12063912717|
|Nonprofits & Acti...|104960.29004804848|
|    Film & Animation|129949.44678126607|
|      People & Blogs|119120.95657795848|
|     News & Politics| 48438.17021188621|
|      Pets & Animals| 71810.13633152505|
|    Autos & Vehicles| 71376.28455411384|
|               Music|219701.52193703968|
|              Comedy|173730.91479345656|
+--------------------+------------------+



# Смотрим на популярные тэги

In [15]:
def split_udf(value, separator):
    @udf(returnType = ArrayType(StringType()))
    def split_udf_inner(value):
        return value.split(separator)
    return split_udf_inner(value)

In [16]:
def split_pudf(value, separator):
    @pandas_udf(ArrayType(StringType()), PandasUDFType.SCALAR)
    def split_pudf_inner(value):
        return value.str.split(pat=separator)
    return split_pudf_inner(value)

In [17]:
def split_scala(tags):
    _split_scala_udf = spark._jvm.CustomUDFs.splitTagsUDF()
    return Column(_split_scala_udf.apply(_to_seq(spark, [tags], _to_java_column)))

In [18]:
tags = videos_sdim_df\
    .select([\
        videos_sdim_df.video_id.alias('tag_video_id'),\
        split(videos_sdim_df.tags, '\|').alias('tags_array'),\
        videos_sdim_df.tags]\
    )\
    .select([\
        col('tag_video_id'),\
        explode(col('tags_array')).alias('tag'),\
        col('tags')\
    ])
tags.show()

+------------+----------------+--------------------+
|tag_video_id|             tag|                tags|
+------------+----------------+--------------------+
| -LoSw4o2zDQ|          recipe|recipe|recipes|ho...|
| -LoSw4o2zDQ|         recipes|recipe|recipes|ho...|
| -LoSw4o2zDQ|     how to make|recipe|recipes|ho...|
| -LoSw4o2zDQ|     how to bake|recipe|recipes|ho...|
| -LoSw4o2zDQ|         cooking|recipe|recipes|ho...|
| -LoSw4o2zDQ|            food|recipe|recipes|ho...|
| -LoSw4o2zDQ|          sweets|recipe|recipes|ho...|
| -LoSw4o2zDQ|         dessert|recipe|recipes|ho...|
| -LoSw4o2zDQ|            cute|recipe|recipes|ho...|
| -LoSw4o2zDQ|            easy|recipe|recipes|ho...|
| -LoSw4o2zDQ|           quick|recipe|recipes|ho...|
| -LoSw4o2zDQ|          kawaii|recipe|recipes|ho...|
| -LoSw4o2zDQ|   kawaii baking|recipe|recipes|ho...|
| -LoSw4o2zDQ|kawaiisweetworld|recipe|recipes|ho...|
| -LoSw4o2zDQ|     kawaii food|recipe|recipes|ho...|
| -LoSw4o2zDQ|       cute food|recipe|recipes|

In [19]:
tags_udf = videos_sdim_df\
    .select([\
        videos_sdim_df.video_id.alias('tag_video_id'),\
        split_udf(videos_sdim_df.tags, '|').alias('tags_array'),\
        videos_sdim_df.tags]\
    )\
    .select([\
        col('tag_video_id'),\
        explode(col('tags_array')).alias('tag'),\
        col('tags')\
    ])
tags_udf.show()

+------------+----------------+--------------------+
|tag_video_id|             tag|                tags|
+------------+----------------+--------------------+
| -LoSw4o2zDQ|          recipe|recipe|recipes|ho...|
| -LoSw4o2zDQ|         recipes|recipe|recipes|ho...|
| -LoSw4o2zDQ|     how to make|recipe|recipes|ho...|
| -LoSw4o2zDQ|     how to bake|recipe|recipes|ho...|
| -LoSw4o2zDQ|         cooking|recipe|recipes|ho...|
| -LoSw4o2zDQ|            food|recipe|recipes|ho...|
| -LoSw4o2zDQ|          sweets|recipe|recipes|ho...|
| -LoSw4o2zDQ|         dessert|recipe|recipes|ho...|
| -LoSw4o2zDQ|            cute|recipe|recipes|ho...|
| -LoSw4o2zDQ|            easy|recipe|recipes|ho...|
| -LoSw4o2zDQ|           quick|recipe|recipes|ho...|
| -LoSw4o2zDQ|          kawaii|recipe|recipes|ho...|
| -LoSw4o2zDQ|   kawaii baking|recipe|recipes|ho...|
| -LoSw4o2zDQ|kawaiisweetworld|recipe|recipes|ho...|
| -LoSw4o2zDQ|     kawaii food|recipe|recipes|ho...|
| -LoSw4o2zDQ|       cute food|recipe|recipes|

In [20]:
tags_pudf = videos_sdim_df\
    .select([\
        videos_sdim_df.video_id.alias('tag_video_id'),\
        split_pudf(videos_sdim_df.tags, '|').alias('tags_array'),\
        videos_sdim_df.tags]\
    )\
    .select([\
        col('tag_video_id'),\
        explode(col('tags_array')).alias('tag'),\
        col('tags')\
    ])
tags_pudf.show()

+------------+----------------+--------------------+
|tag_video_id|             tag|                tags|
+------------+----------------+--------------------+
| -LoSw4o2zDQ|          recipe|recipe|recipes|ho...|
| -LoSw4o2zDQ|         recipes|recipe|recipes|ho...|
| -LoSw4o2zDQ|     how to make|recipe|recipes|ho...|
| -LoSw4o2zDQ|     how to bake|recipe|recipes|ho...|
| -LoSw4o2zDQ|         cooking|recipe|recipes|ho...|
| -LoSw4o2zDQ|            food|recipe|recipes|ho...|
| -LoSw4o2zDQ|          sweets|recipe|recipes|ho...|
| -LoSw4o2zDQ|         dessert|recipe|recipes|ho...|
| -LoSw4o2zDQ|            cute|recipe|recipes|ho...|
| -LoSw4o2zDQ|            easy|recipe|recipes|ho...|
| -LoSw4o2zDQ|           quick|recipe|recipes|ho...|
| -LoSw4o2zDQ|          kawaii|recipe|recipes|ho...|
| -LoSw4o2zDQ|   kawaii baking|recipe|recipes|ho...|
| -LoSw4o2zDQ|kawaiisweetworld|recipe|recipes|ho...|
| -LoSw4o2zDQ|     kawaii food|recipe|recipes|ho...|
| -LoSw4o2zDQ|       cute food|recipe|recipes|

In [21]:
tags_scala = videos_sdim_df\
    .select([\
        videos_sdim_df.video_id.alias('tag_video_id'),\
        split_scala(videos_sdim_df.tags).alias('tags_array'),\
        videos_sdim_df.tags]\
    )\
    .select([\
        col('tag_video_id'),\
        explode(col('tags_array')).alias('tag'),\
        col('tags')\
    ])
tags_scala.show()

+------------+----------------+--------------------+
|tag_video_id|             tag|                tags|
+------------+----------------+--------------------+
| -LoSw4o2zDQ|          recipe|recipe|recipes|ho...|
| -LoSw4o2zDQ|         recipes|recipe|recipes|ho...|
| -LoSw4o2zDQ|     how to make|recipe|recipes|ho...|
| -LoSw4o2zDQ|     how to bake|recipe|recipes|ho...|
| -LoSw4o2zDQ|         cooking|recipe|recipes|ho...|
| -LoSw4o2zDQ|            food|recipe|recipes|ho...|
| -LoSw4o2zDQ|          sweets|recipe|recipes|ho...|
| -LoSw4o2zDQ|         dessert|recipe|recipes|ho...|
| -LoSw4o2zDQ|            cute|recipe|recipes|ho...|
| -LoSw4o2zDQ|            easy|recipe|recipes|ho...|
| -LoSw4o2zDQ|           quick|recipe|recipes|ho...|
| -LoSw4o2zDQ|          kawaii|recipe|recipes|ho...|
| -LoSw4o2zDQ|   kawaii baking|recipe|recipes|ho...|
| -LoSw4o2zDQ|kawaiisweetworld|recipe|recipes|ho...|
| -LoSw4o2zDQ|     kawaii food|recipe|recipes|ho...|
| -LoSw4o2zDQ|       cute food|recipe|recipes|

## Время выполнения верхних блоков

Отображается в ноутбуке после отработки блока в vscode

| вариант | время в секундах | повторный перезапуск |
| --: | --:| --: |
| split spark| 0.3 | 0.2 |
| split udf| 0.4 | 0.3
| split pandas udf| 0.5 | 0.5 |
| split scala | 0.3 | 0.2 |

In [22]:
popular_tags = tags\
    .filter(col('tag') != '[none]')\
    .groupBy('tag')\
    .agg(
        countDistinct('tag_video_id').alias('video_count')
    )\
    .sort(col('video_count').desc())
popular_tags.show()

+-----------+-----------+
|        tag|video_count|
+-----------+-----------+
|      funny|        217|
|     comedy|        163|
|       2017|         93|
|      humor|         92|
|     how to|         84|
|     makeup|         77|
|      music|         74|
|       vlog|         73|
|      video|         71|
|   tutorial|         69|
|  interview|         69|
|  celebrity|         64|
|     review|         61|
|       news|         61|
|celebrities|         59|
|     beauty|         58|
|       food|         57|
|    science|         56|
|   comedian|         55|
|funny video|         54|
+-----------+-----------+
only showing top 20 rows



# Выбираем топ-5 комментариев по кошачьим видео

In [23]:
cats_top5_df = videos_sdim_df\
    .select(['video_id', col('title').alias('video_title')])\
    .join(tags.filter(col('tag') == 'cat').distinct(), videos_sdim_df.video_id == tags.tag_video_id, 'inner')\
    .join(comments.withColumnRenamed('video_id', 'comment_video_id'), col('comment_video_id') == videos_sdim_df.video_id, 'inner')\
    .select([
        col('video_title'),\
        col('comment_text'),\
        col('likes'),\
        row_number().over(Window.partitionBy(col('video_id')).orderBy(col('likes').desc())).alias('row_number')
    ])\
    .filter(col('row_number') <= 5)
cats_top5_df.show()

+--------------------+--------------------+-----+----------+
|         video_title|        comment_text|likes|row_number|
+--------------------+--------------------+-----+----------+
|7. Teach the Firs...|Cannot hear you, ...|    0|         1|
|Cat vs Dog - Best...|The second I read...| 2355|         1|
|Cat vs Dog - Best...|talk about the oc...| 1070|         2|
|Cat vs Dog - Best...|talk about the oc...| 1021|         3|
|Cat vs Dog - Best...|talk about the oc...|  957|         4|
|Cat vs Dog - Best...|talk about the oc...|  802|         5|
|Things you didn't...|Simon, I thought ...|    8|         1|
|Things you didn't...|My cat loves stri...|    6|         2|
|Things you didn't...|My cat loves stri...|    6|         3|
|Things you didn't...|My cats 17 and st...|    6|         4|
|Things you didn't...|My cats 17 and st...|    6|         5|
| MEET MY NEW KITTEN!|I can’t believe h...|    4|         1|
| MEET MY NEW KITTEN!|i want colleen to...|    2|         2|
| MEET MY NEW KITTEN!|Th